In [5]:
"""
Challenge #4
Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
체인을 수동으로 구현해야 합니다.
체인에 ConversationBufferMemory를 부여합니다.
이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
체인에 다음 질문을 합니다:
Aaronson 은 유죄인가요?
그가 테이블에 어떤 메시지를 썼나요?
Julia 는 누구인가요?

Hint.
다음과 같은 절차대로 구현하면 챌린지를 해결할 수 있습니다.
(1) 문서 로드하기 : TextLoader 등 을 사용해서 파일에서 텍스트를 읽어옵니다. (Document Loaders 관련 문서
(2) 문서 쪼개기 : CharacterTextSplitter 등 을 사용해서 문서를 작은 문서 조각들로 나눕니다. Character Split 관련 문서
(3) 임베딩 생성 및 캐시 : OpenAIEmbeddings, CacheBackedEmbeddings 등 을 사용해 문서 조각들을 임베딩하고 임베딩을 저장합니다. Caching 관련 문서
(4) 벡터 스토어 생성 : FAISS 등 을 사용해서 임베딩된 문서들을 저장하고 검색할 수 있는 데이터베이스를 만듭니다. FAISS 관련 문서
(5) 대화 메모리와 질문 처리 : ConversationBufferMemory를 사용해 대화 기록을 관리합니다.
(6) 체인 연결 : 앞에서 구현한 컴포넌트들을 적절하게 체인으로 연결합니다.
"""
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import  LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/challenge4_doc.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

#vectorstore = Chroma.from_documents(docs, embeddings)
vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Aaronson 은 유죄인가요?")

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

'네, Jones, Aaronson 및 Rutherford은 자신들에게 적혀진 범죄로 유죄 판결을 받았습니다.'

In [6]:
chain.run("그가 테이블에 어떤 메시지를 썼나요?")#

'그가 테이블에 어떤 메시지를 썼는지에 대한 정보는 제공되지 않았습니다.'

In [7]:
chain.run("Julia 는 누구인가요?") #

'Julia는 "1984"라는 조지 오웰의 소설에서 주인공 윈스턴 스미스가 사랑하는 여성 캐릭터입니다. 그녀는 윈스턴과 함께 서로를 사랑하면서 정부의 통제와 감시에 맞서 싸우는 중요한 역할을 합니다.'